In [1]:
import os

import boto3
import pandas as pd
import pyspark

In [2]:
session = boto3.Session(
    aws_access_key_id = os.environ.get("MINIO_ROOT_USER"),
    aws_secret_access_key = os.environ.get("MINIO_ROOT_PASSWORD"),
)
s3 = session.resource('s3', 
  endpoint_url="http://minio:9000", 
  config=boto3.session.Config(signature_version='s3v4')
)
s3.create_bucket(Bucket='spark-data')

s3.Bucket(name='spark-data')

In [37]:
sc.stop()

In [38]:
conf = pyspark.SparkConf().setMaster("spark://spark-master:7077")
conf.set('spark.hadoop.fs.s3a.endpoint', 'http://minio:9000')
conf.set('spark.hadoop.fs.s3a.access.key', os.environ.get("MINIO_ROOT_USER"))
conf.set('spark.hadoop.fs.s3a.secret.key', os.environ.get("MINIO_ROOT_PASSWORD"))
conf.set('spark.hadoop.fs.s3a.path.style.access', "true")
conf.set("spark.jars.packages", "io.delta:delta-core_2.12-3.2.1")
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
sc = pyspark.SparkContext(conf=conf)

In [3]:
spark.stop()

In [2]:
spark = pyspark.sql.SparkSession.builder. \
  config('spark.hadoop.fs.s3a.endpoint', 'http://minio:9000'). \
  config('spark.hadoop.fs.s3a.access.key', os.environ.get("MINIO_ROOT_USER")). \
  config('spark.hadoop.fs.s3a.secret.key', os.environ.get("MINIO_ROOT_PASSWORD")). \
  config('spark.hadoop.fs.s3a.path.style.access', "true"). \
  config('spark.jars', "/home/jptr/app/delta-spark_2.12-3.2.0.jar,/opt/spark_jars/hadoop-aws-3.3.4.jar"). \
  config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0"). \
  config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension"). \
  master("spark://spark-master:7077").getOrCreate()
  

:: loading settings :: url = jar:file:/home/jptr/.local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jptr/.ivy2/cache
The jars for the packages stored in: /home/jptr/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6cd25f82-6e72-4bee-8fa2-9edd1a9b2b36;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.2.0/delta-spark_2.12-3.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.2.0!delta-spark_2.12.jar (1795ms)
:: resolution report :: resolve 596ms :: artifacts dl 1801ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifact

In [3]:
transactions = pd.DataFrame({
    "id": [i for i in range(10)],
    "date": ["2024-01-01"] * 4 + ["2024-02-01"] * 6,
    "cust_id": [0, 0, 0, 1, 1, 2, 2, 2, 3, 3],
    "prod_id": [0, 1, 2, 0, 1, 2, 2, 3, 0, 3],
    "volume": [1, 1, 1, 1, 2, 2, 3, 3, 4, 4],
    "revenue": [10, 15, 40, 10, 30, 80, 120, 150, 40, 160]
}).set_index("id")

In [4]:
transactions_spark = spark.createDataFrame(transactions)

In [5]:
transactions_spark.show()

[Stage 0:>                                                          (0 + 1) / 1]24/11/28 13:54:43 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (172.21.0.6 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 8) than that in driver 3.11, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIter

Py4JJavaError: An error occurred while calling o62.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (172.21.0.6 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 8) than that in driver 3.11, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.

In [ ]:
db_url = "jdbc:postgresql://demo_db:5432/demo_db"
properties = {
    "user": "user",
    "password": "shhh_its_a_secret",
    "driver": "org.postgresql.Driver"
}